captsone interpreter
https://www.capstone-engine.org/

In [2]:
%%file /tmp/test.c

int myadd(int a, int b) {
    return a + b;
}


Writing /tmp/test.c


In [3]:
! gcc -c -o /tmp/test.so /tmp/test.c

In [4]:
from capstone import *

CODE = b"\x55\x48\x8b\x05\xb8\x13\x00\x00"

md = Cs(CS_ARCH_X86, CS_MODE_64)
for i in md.disasm(CODE, 0x1000):
    print("0x%x:\t%s\t%s" %(i.address, i.mnemonic, i.op_str))


0x1000:	push	rbp
0x1001:	mov	rax, qword ptr [rip + 0x13b8]


In [ ]:
from capstone import *

CODE = b"\x55\x48\x8b\x05\xb8\x13\x00\x00"
from collections import defaultdict
class State():
    mem = defaultdict(None)
    reg = defaultdict(None)

def interp(op, state):
    
    match op.mnemonic:
        case "add":
            state.reg[op.operands[0].reg] += state[op.operands[1].reg]
        case "sub":
            state[op.operands[0].reg] -= state[op.operands[1].reg]
        case "mov":
            if op.operands[0].type == X86_OP_IMM:
                state[op.operands[0].reg] = op.operands[1].imm
            elif op.operands[0].type == X86_OP_REG:
                op.operands[0].reg = op.operands[1].reg
            elif op.operands[0].type == X86_OP_MEM:
                state[op.operands[0].reg] = mem[state[op.operands[1].reg + op.operands[1].disp]
    return state
state = defaultdict(None)
for i in md.disasm(CODE, 0x1000):
    print(dir(i))
    interp(i, state)


['_CsInsn__gen_detail', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattr__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_cs', '_raw', 'address', 'bytes', 'errno', 'group', 'group_name', 'groups', 'id', 'insn_name', 'mnemonic', 'op_count', 'op_find', 'op_str', 'reg_name', 'reg_read', 'reg_write', 'regs_access', 'regs_read', 'regs_write', 'size']
['_CsInsn__gen_detail', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattr__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_cs',

CsError: Details are unavailable (CS_ERR_DETAIL)

Ok so it gets hairier and harrier

Getting the smeanuitcs



In [ ]:
import pypcode
class State():
    space = defaultdict(defaultdict)

def interp(insn, state):
    match insn.op:
        case pypcode.PcodeOp.INT_ADD:
            
